In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [2]:
import os
import json
import random
import numpy as np
# from PIL import Image # ★ PIL 제거
import cv2 # ★ OpenCV 임포트 추가

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from tqdm import tqdm
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # ToTensorV2 임포트 추가 (관례상)

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ================================
# 3. Transform / Dataset / DataLoader (★ Albumentations로 통일 ★)
# ================================
# ResNet 기본 입력 크기 224x224 기준

# ★★★ 수정 1: transform -> train_transform 이름 변경 및 A.Compose 유지 ★★★
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

# ★★★ 수정 2: val_transform도 A.Compose로 통일 (증강 기법 제거) ★★★
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])


# ★ 경로 확인! ImageFolder의 루트는 'cropped' 폴더가 되어야 합니다. ★
data_root = "/workspace/user4/cropped"

# ImageFolder 사용: 폴더 이름(anger, happy, panic, sadness)이 클래스 라벨이 됩니다.
train_full_dataset = datasets.ImageFolder(os.path.join(data_root, "train"), transform=None) # transform은 CustomBinaryDataset에서 직접 처리
val_full_dataset = datasets.ImageFolder(os.path.join(data_root, "val"), transform=None) # transform은 CustomBinaryDataset에서 직접 처리

# ImageFolder는 모든 폴더를 클래스로 인식합니다.
# "invalid" 폴더는 제외하고 "happy" vs "others" 2진 분류를 위한 데이터셋 필터링 필요
class_to_idx = train_full_dataset.class_to_idx
target_classes = ["anger", "happy", "panic", "sadness"]

# ⭐️ 2진 분류를 위한 데이터 필터링 ⭐️
def filter_and_remap_dataset(full_dataset, transform, target_classes): # transform 인자 그대로 사용
    
    # 0: OTHERS (anger, panic, sadness), 1: HAPPY
    new_samples = []
    # ImageFolder의 samples는 (파일 경로, 인덱스) 튜플입니다.
    for path, class_idx in full_dataset.samples:
        class_name = full_dataset.classes[class_idx]
        
        # 'invalid' 폴더에 속하는 파일은 건너뜁니다.
        if class_name not in target_classes:
            continue
            
        # 2진 분류 라벨 할당
        if class_name == "happy":
            binary_label = 1 # HAPPY
        else:
            binary_label = 0 # OTHERS (anger, panic, sadness)
            
        new_samples.append((path, binary_label))
        
    # PyTorch 기본 Dataset으로 대체
    class CustomBinaryDataset(Dataset):
        def __init__(self, samples, transform):
            self.samples = samples
            self.transform = transform
            
        def __len__(self):
            return len(self.samples)
            
        def __getitem__(self, idx):
            path, label = self.samples[idx]
            
            # ★ OpenCV로 이미지 로드 ★
            image = cv2.imread(path)
            
            # 파일이 없거나 유효하지 않은 경우 대비
            if image is None:
                 raise FileNotFoundError(f"Image not found or invalid: {path}")

            # OpenCV는 기본적으로 BGR 순서로 로드하므로, RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환

            if self.transform is not None:
                # Albumentations 적용 방식 사용 (A.Compose 전달 시)
                augmented = self.transform(image=image)
                image = augmented['image'] # ToTensorV2로 인해 이미 텐서로 변환됨
            
            # 라벨은 torch.long 타입으로 반환
            return image, torch.tensor(label, dtype=torch.long)

    print(f"Original {len(full_dataset)} samples reduced to {len(new_samples)} (excluding invalid).")
    # full_dataset.transform 대신 새로 정의된 transform 사용
    return CustomBinaryDataset(new_samples, transform) 


# ★★★ 수정 3: filter_and_remap_dataset 호출 시 train_transform/val_transform 전달 ★★★
train_dataset = filter_and_remap_dataset(train_full_dataset, train_transform, target_classes) 
val_dataset = filter_and_remap_dataset(val_full_dataset, val_transform, target_classes) 


batch_size = 32
# num_workers=4는 환경에 따라 에러가 발생할 수 있어 0 또는 2로 조정하는 것이 안전할 수 있습니다.
# num_workers = 4 
num_workers = 2 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print("\n✅ 전처리 및 데이터 로드 완료 (OpenCV + Albumentations). 다음 블록으로 진행하세요.")
# ================================
# 4. 모델 정의 (ResNet18, 2-class) - 변경 없음
# ...
# ================================
# 사전학습된 resnet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 fc를 2클래스로 교체 (HAPPY vs OTHERS)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)
# 가중치 없는 기본 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수 - 변경 없음
# ...
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        # ⭐️ torch.inference_mode() 적용 ⭐️
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_resnet_happy_vs_others.pth")
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

# # ⭐️ torch.inference_mode() 적용 ⭐️
with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))

device: cuda
Original 9550 samples reduced to 9535 (excluding invalid).
Original 1151 samples reduced to 1149 (excluding invalid).
Train samples: 9535
Val samples: 1149

✅ 전처리 및 데이터 로드 완료 (OpenCV + Albumentations). 다음 블록으로 진행하세요.

Epoch 1/10
------------------------------


100%|██████████| 298/298 [00:56<00:00,  5.31it/s]

Train Loss: 0.1455 Acc: 0.9455


Val   Loss: 0.2071 Acc: 0.9312
>> best model updated

Epoch 2/10
------------------------------


100%|██████████| 298/298 [00:57<00:00,  5.23it/s]

...

Train Loss: 0.0115 Acc: 0.9962


Val   Loss: 0.2065 Acc: 0.9443

Epoch 10/10
------------------------------


100%|██████████| 298/298 [00:36<00:00,  8.24it/s]

Train Loss: 0.0108 Acc: 0.9962


Val   Loss: 0.0973 Acc: 0.9756
>> best model updated

Best val Acc: 0.9756

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9756
Precision: 0.9357
Recall   : 0.9732
F1-score : 0.9541

Confusion Matrix (rows: true, cols: pred)
[[830  20]
 [  8 291]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.99      0.98      0.98       850
       HAPPY       0.94      0.97      0.95       299

    accuracy                           0.98      1149
   macro avg       0.96      0.97      0.97      1149
weighted avg       0.98      0.98      0.98      1149



In [1]:
import torch
import torch.nn as nn
from torchvision import models
from thop import profile
# copy는 model 정의 시 필요하지 않으므로 제외

# 1. 초기 설정 재확인 (이전 출력에서 device가 'cuda'로 설정되었음을 확인했습니다.)
# 만약 'device'가 정의되지 않았다면, 이 라인도 다시 실행해야 합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 2. 모델 정의 및 최적 가중치 로드
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 fc를 2클래스로 교체
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# 모델을 디바이스로 이동
model = model.to(device)

# 학습된 최적 가중치 로드 (best_resnet_happy_vs_others.pth 파일이 존재한다고 가정)
try:
    model.load_state_dict(torch.load("best_resnet_happy_vs_others.pth"))
    print("✅ 최적 가중치 로드 완료.")
except FileNotFoundError:
    print("🚨 경고: 'best_resnet_happy_vs_others.pth' 파일을 찾을 수 없습니다. 모델 가중치가 랜덤 상태일 수 있습니다.")

model.eval() # 평가 모드로 설정

# 3. FLOPs 및 파라미터 수 계산
# 224x224 크기의 더미 입력 텐서를 생성합니다.
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# FLOPs와 파라미터 수를 계산합니다.
total_flops, total_params = profile(model, inputs=(dummy_input,), verbose=False)

print(f"\n=== 계산된 연산량 및 파라미터 수 (ResNet18) ===")
print(f"Total Parameters (가중치 수): {total_params / 1e6:.2f} M") # M = Million
print(f"Total FLOPs (연산 횟수): {total_flops / 1e9:.2f} G") # G = Giga

device: cuda
✅ 최적 가중치 로드 완료.

=== 계산된 연산량 및 파라미터 수 (ResNet18) ===
Total Parameters (가중치 수): 11.18 M
Total FLOPs (연산 횟수): 1.82 G


In [3]:
import torch
import torchvision.models as models
import time
import statistics # 표준 편차 계산을 위해 추가

# 1. 장치 설정 (GPU 사용 가능 여부 확인)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 사용 장치: {device}")
print("-" * 30)

# 2. ResNet-18 모델 로드 및 준비
# pretrained=True 대신 최신 weights=... 방식으로 로드합니다.
# model 변수에 ResNet-18 모델 객체가 할당됩니다.
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 3. 모델을 평가 모드(eval)로 설정 (필수)
# Dropout, BatchNorm 등이 추론 모드로 작동하도록 합니다.
model.eval()

# 4. 모델을 지정된 장치(GPU 또는 CPU)로 이동
model = model.to(device)

# 5. Dummy Input 정의
# (Batch size, Channel, Height, Width) = (1, 3, 224, 224)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

print("✅ ResNet-18 모델 및 Dummy Input 준비 완료.")
print(f"입력 데이터 형태: {dummy_input.shape}")
print("-" * 30)

# 6. 추론 속도 측정 시작
ITERATIONS = 100

print(f"=== 추론 속도 측정 시작 (Warm-up 10회, 측정 {ITERATIONS}회) ===")

# --- Warm-up (워밍업) ---
# 첫 연산은 로딩 등으로 느릴 수 있으므로 10회 미리 실행합니다.
for _ in range(10):
    with torch.no_grad():
        _ = model(dummy_input)

# GPU 연산 동기화 (필수)
# GPU 연산은 비동기적으로 수행되므로, 타이머를 시작하기 전에 이전 연산이 모두 완료되도록 동기화합니다.
if device.type == 'cuda':
    torch.cuda.synchronize()

# --- 실제 측정 ---
timings = []

if device.type == 'cuda':
    # GPU 측정 시에는 CUDA Event를 사용합니다.
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    with torch.no_grad():
        for rep in range(ITERATIONS):
            starter.record()
            _ = model(dummy_input)
            ender.record()
            
            # 동기화 및 시간 기록
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)  # 밀리초(ms) 단위
            timings.append(curr_time)
            
else:
    # CPU 측정 시에는 time 모듈을 사용합니다.
    with torch.no_grad():
        for rep in range(ITERATIONS):
            start_time = time.time()
            _ = model(dummy_input)
            end_time = time.time()
            curr_time = (end_time - start_time) * 1000 # 초(s)를 밀리초(ms)로 변환
            timings.append(curr_time)

# 결과 출력
mean_syn = statistics.mean(timings)
std_syn = statistics.stdev(timings) if len(timings) > 1 else 0

print(f"\n--- 결과 ---")
print(f"평균 추론 시간 ({ITERATIONS}회): {mean_syn:.3f} ms")
print(f"표준 편차: {std_syn:.3f} ms")

✅ 사용 장치: cuda
------------------------------
✅ ResNet-18 모델 및 Dummy Input 준비 완료.
입력 데이터 형태: torch.Size([1, 3, 224, 224])
------------------------------
=== 추론 속도 측정 시작 (Warm-up 10회, 측정 100회) ===

--- 결과 ---
평균 추론 시간 (100회): 1.944 ms
표준 편차: 0.036 ms


In [5]:
import torch
from tabulate import tabulate # 표 형식 출력을 위해 tabulate 라이브러리 사용 (설치 필요: pip install tabulate)
import statistics # 표준 편차 계산을 위해 필요

# --- 1. 이전 측정 결과 불러오기 (이전 코드에서 계산된 변수 사용) ---
# 다음 변수들이 이전 속도 측정 코드에서 정의되었다고 가정합니다:
# mean_syn (평균 추론 시간, ms)
# std_syn (표준 편차, ms)
# device (사용 장치, 'cuda' 또는 'cpu')

# --- 2. ResNet-18 (ImageNet 분류용)의 고정 지표 설정 ---
# ResNet-18의 파라미터 및 FLOPs 값 (224x224 입력 기준)
PARAM_M = 11.69  # 11.69 Million Parameters
FLOPS_B = 1.81   # 1.81 Billion FLOPs (224x224)

# 주의: ResNet-18은 분류 모델이므로 mAP (탐지 정확도) 대신 
# ImageNet Top-1 Accuracy를 참고 지표로 사용합니다.
ACCURACY_METRIC = "69.7%" 

# --- 3. 표 데이터 구성 ---
# 표의 헤더 (CPU 속도 제외)
headers = [
    "모델", 
    "크기 (픽셀)", 
    "mAPᵥᵃˡ 50-95 (참고:분류)", 
    f"속도 {device.type.upper()} (ms)", 
    "파라미터 (M)", 
    "FLOPs (B)"
]

# 측정 결과 포맷: 평균 ± 표준편차 (YOLO 표 형식과 일치하도록)
speed_output = f"{mean_syn:.2f} ± {std_syn:.2f}"

# 표의 데이터 행
# ResNet-18의 기본 입력 크기는 224x224이지만, YOLO와 비교를 위해 640 대신 224로 표시합니다.
table_data = [
    ["ResNet18", "224", ACCURACY_METRIC, speed_output, f"{PARAM_M:.1f}", f"{FLOPS_B:.1f}"]
]

# --- 4. 표 출력 ---
print("\n" + "=" * 80)
print("=== ResNet-18 성능 요약 (YOLO 표 형식 참고) ===")
print("  *주의: ResNet18은 이미지 분류 모델이므로 mAP 대신 ImageNet Top-1 정확도를 사용합니다.")
print(tabulate(table_data, headers=headers, tablefmt="github"))
print("=" * 80)


=== ResNet-18 성능 요약 (YOLO 표 형식 참고) ===
  *주의: ResNet18은 이미지 분류 모델이므로 mAP 대신 ImageNet Top-1 정확도를 사용합니다.
| 모델     |   크기 (픽셀) | mAPᵥᵃˡ 50-95 (참고:분류)   | 속도 CUDA (ms)   |   파라미터 (M) |   FLOPs (B) |
|----------|---------------|----------------------------|------------------|----------------|-------------|
| ResNet18 |           224 | 69.7%                      | 1.94 ± 0.04      |           11.7 |         1.8 |


In [ ]:
import torch
from tabulate import tabulate
import statistics 
import numpy as np # 파라미터/FLOPs 지표 출력을 위해 필요

# --- 1. 이전 측정 결과 불러오기 (이전 코드에서 계산된 변수 사용 가정) ---
# 다음 변수들이 이전 속도 측정 코드에서 정의되었다고 가정합니다:
# mean_syn (평균 추론 시간, ms)
# std_syn (표준 편차, ms)
# device (사용 장치, 'cuda' 또는 'cpu')

# 만약 이전에 측정한 mean_syn, std_syn이 없다면 임시 값을 설정하여 실행해야 합니다.
try:
    mean_syn = mean_syn
    std_syn = std_syn
    device = device
except NameError:
    print("경고: mean_syn, std_syn, device 변수가 정의되지 않아 임시 값으로 대체합니다. 속도 측정 코드를 먼저 실행하세요.")
    mean_syn = 5.0
    std_syn = 0.5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# --- 2. ResNet-18 (ImageNet 분류용)의 고정 지표 설정 ---
# ResNet-18의 파라미터 및 FLOPs 값 (224x224 입력 기준)
PARAM_M = 11.69  # Million Parameters
FLOPS_B = 1.81   # Billion FLOPs

# ResNet-18의 ImageNet Top-1 및 Top-5 정확도
TOP1_ACC = "69.7%"
TOP5_ACC = "89.0%"
ACCURACY_METRIC = f"{TOP1_ACC} (Top-1) / {TOP5_ACC} (Top-5)"

# --- 3. 표 데이터 구성 ---
# 표의 헤더 (CPU 속도 제외)
headers = [
    "모델", 
    "크기 (픽셀)", 
    "정확도 (ImageNet)", 
    f"속도 {device.type.upper()} (ms)", 
    "파라미터 (M)", 
    "FLOPs (B)"
]


=== ResNet-18 성능 요약 (YOLO 표 형식 참고) ===
  *참고: ResNet18은 이미지 분류 모델이므로 mAP 대신 ImageNet Top-1/Top-5 정확도를 사용합니다.
| 모델     |   크기 (픽셀) | 정확도 (ImageNet)             | 속도 CUDA (ms)   |   파라미터 (M) |   FLOPs (B) |
|----------|---------------|-------------------------------|------------------|----------------|-------------|
| ResNet18 |           224 | 69.7% (Top-1) / 89.0% (Top-5) | 1.94 ± 0.04      |           11.7 |         1.8 |
